In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('word_tokenize')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [ ]:
import pandas as pd
df=pd.read_csv('all training labeled data_social media - hsbc_twitter_DATA_not GENERAL.csv', usecols=['tweet'])
df.head()

,tweet
0,"@LondOgrad Hello Eugene, we do apologise for t..."
1,@ncohen4 @CooperativeBank @HSBC @HSBCUKBusines...
2,"@sharaola Good morning, sorry for your experie..."
3,@Bounce_BackLoan Still not a peep from HSBC
4,@djm2481 @mgcmitchell @HSBC_UK Thanks for the ...


In [ ]:
df.shape

(4467, 1)

In [ ]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
df['tweet'].apply(lambda x: remove_emoji(x))

0       @LondOgrad Hello Eugene, we do apologise for t...
1       @ncohen4 @CooperativeBank @HSBC @HSBCUKBusines...
2       @sharaola Good morning, sorry for your experie...
3             @Bounce_BackLoan Still not a peep from HSBC
4       @djm2481 @mgcmitchell @HSBC_UK Thanks for the ...
                              ...                        
4462    @demosisto why should HSBC show their support?...
4463    @BritishBBank @Bounce_BackLoan @Metro_Bank I w...
4464    @WWIIadvertising Hi David. Sorry somethings up...
4465    @HSBC thank you I just called in on the phone ...
4466    @shezRommy @Bounce_BackLoan Probably be quicke...
Name: tweet, Length: 4467, dtype: object

In [ ]:
def get_stopwords():
    stop_words = stopwords.words('english')
    stop_words.extend(['hsbc','lloyds','santander','barclays','bank','hdfc','ac','bademjaan','dtype','eugene','foggysmile','follow','hello','londograd','martinslewis','name','quicke','shezrommy','thot','tweet','hsbcukbusiness','ocvbtjrbz','http','see','get','amp','like','line','may','short','ubs','today','make','know','people','say','take','day','also','point','one','even','use','try','kindly','would','rishisunak','fromthequizmaster','please','sorry','full','send','number','much','big','new','able','far','let','via','want','jpm','ita','dona'])
    return stop_words

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus.reader.wordnet import WordNetError

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:

def get_wordnet_pos(word):
    '''
    tags parts of speech to tokens
    Expects a string and outputs the string and its part of speech
    '''

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def word_lemmatizer(text):
    '''
    lemamtizes the tokens based on their part of speech
    Expects a lits of tokens and outputs a list of lemmatized tokens
    '''

    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text, get_wordnet_pos(text))
    return text

In [ ]:
def reflection_tokenizer(text):
    '''
    Tokenizes a list of string, expects a list of strings and outputs a list of strings.
    before tokenizing:
    1)removes the non-alphanumeric charcaters like emojies
    2)removes the numbers
    3)lower cases the words
    4)tokenizes the sentences
    5)lemmatizes teh tokens
    6)removes the tokens in stop words list
     '''

    text=re.sub(r'[\W_]+', ' ', text) #keeps just alphnumeric characters
    text=re.sub(r'\d+', '', text) #removes numbers
    text=text.lower()
    tokens = [word for word in word_tokenize(text)]
    tokens = [word for word in tokens if len(word) >= 3]#removes smaller than 3 character
    tokens = [word_lemmatizer(w) for w in tokens]
    tokens = [s for s in tokens if s not in get_stopwords()]
    return tokens

In [ ]:

docs=df['tweet'].apply(lambda x: reflection_tokenizer(x))

In [ ]:
print(docs[:5])

0    [apologise, delay, due, current, situation, hu...
1    [ncohen, cooperativebank, whole, situation, me...
2    [sharaola, good, morning, experience, contact,...
3                      [bounce, backloan, still, peep]
4    [djm, mgcmitchell, thanks, mention, paul, happ...
Name: tweet, dtype: object


In [ ]:
docs=docs.apply(str)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

token_vectorizer = CountVectorizer(tokenizer = reflection_tokenizer, stop_words=stop_words, ngram_range=(1, 3))
X = token_vectorizer.fit_transform(docs)


In [ ]:
type(X)

scipy.sparse.csr.csr_matrix

In [ ]:
X.shape

(4467, 92878)

In [ ]:
[id2word.doc2bow(text) for text in texts]

In [ ]:

!pip install guidedlda

In [ ]:
import guidedlda
tf_feature_names = token_vectorizer.get_feature_names()
word2id = dict((v, idx) for idx, v in enumerate(tf_feature_names))

In [ ]:
word2id

{'aaachatterjee': 0,
 'aaachatterjee sharmapayal': 1,
 'aaachatterjee sharmapayal payalkamat': 2,
 'aaah': 3,
 'aaah beautiful': 4,
 'aaah beautiful visit': 5,
 'aadersh': 6,
 'aadersh raj': 7,
 'aadersh raj mahi': 8,
 'aadityamehrotr': 9,
 'aadityamehrotr abha': 10,
 'aadityamehrotr abha mishra': 11,
 'aajtak': 12,
 'aajtak bseindia': 13,
 'aakansha': 14,
 'aakansha diksha': 15,
 'aakansha diksha merinvarghs': 16,
 'aakash': 17,
 'aakash beast': 18,
 'aakash beast friend': 19,
 'aakash beast lavishpatni': 20,
 'aakash beast satishsathwara': 21,
 'aanchalsweet': 22,
 'aanchalsweet jaydipm': 23,
 'aanchalsweet jaydipm thnovember': 24,
 'aapl': 25,
 'aaravsingh': 26,
 'aaravsingh aaachatterjee': 27,
 'aaravsingh aaachatterjee sharmapayal': 28,
 'aaravsingh akahkr': 29,
 'aaravsingh akahkr sunildsouza': 30,
 'aaravsingh akahkr wasimkh': 31,
 'aaravsingh ice': 32,
 'aaravsingh ice man': 33,
 'aaravsingh joydiphazarika': 34,
 'aaravsingh joydiphazarika jinijony': 35,
 'aaravsingh sandeeppal

In [ ]:
#with the list approach where the model is checking for the words in the word2id dictionary and then assigning groups to words

seed_topic_list=[[ 'bounceback','apply', 'loan','mortgage','pending','application', 'feeder','open','account','bbl'],

               ['phone','call','time', 'long', 'help', 'email', 'heard', 'ago','hold', 'wait',
               'credit card', 'credit', 'card','approval','payment','transaction','fraud','still',
               'customer','service', 'reply','customer service'],

               ['awful','disgrace','disappoint','bad','disappointed','poor'],

                ['great', 'thank','thanks', 'good', 'job'],

               [ 'coronavirus', 'virus','pandemic','lockdown','crisis','impact','cut'],

               ['change','leave', 'switch', 'another','away','rethink','move'],

]

In [ ]:
model = guidedlda.GuidedLDA(n_topics=6, n_iter=100, random_state=7, refresh=10)
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

In [ ]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
     topic_words = np.array(tf_feature_names)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: bounce backloan account bounce backloan apply back loan feeder wait week
Topic 1: customer call account banking back card thanks help service time
Topic 2: thanks thank team link share help check message card code
Topic 3: bounce bounce backloan backloan email account application sign back apply business
Topic 4: gold thanks covid chat live year support global market singapore
Topic 5: tata crucible crucible tata tccq crucible tccq contest tata crucible tccq tccq contest contestalert tccq contest contestalert


In [ ]:
!pip install pyLDAvis==2.1.2


     |████████████████████████████████| 1.6MB 4.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=124867be2deac1ed175853faf6ecd361465fdc79f47af67088060458f891aafe
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
panel = pyLDAvis.sklearn.prepare(model, X, token_vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0       1.465019 -47.625065       1        1  21.701078
3      54.854797 -36.476795       2        1  19.094564
2      32.212929 -61.444572       3        1  16.389289
1       5.111096 -14.110332       4        1  15.630880
4      26.350170 -33.374702       5        1  14.871066
5      38.109818  -7.222781       6        1  12.313125, topic_info=                  Term         Freq        Total Category  logprob  loglift
13533           bounce  1351.000000  1351.000000  Default  30.0000  30.0000
13557  bounce backloan  1083.000000  1083.000000  Default  29.0000  29.0000
8934          backloan  1084.000000  1084.000000  Default  28.0000  28.0000
80154    tata crucible   239.000000   239.000000  Default  27.0000  27.0000
80153             tata   239.000000   239.000000  Default  26.0000  26.0000
...                ...          ...          ...     

In [ ]:
pyLDAvis.display(panel)